## 0. Load Dataset

In [19]:
from datasets import load_dataset

In [20]:
cache_dir = "/ocean/projects/cis250208p/shared/datasets"

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ccvl/3DSRBench", cache_dir=cache_dir)

In [21]:
print(ds)

DatasetDict({
    test: Dataset({
        features: ['index', 'question', 'A', 'B', 'C', 'D', 'answer', 'category', 'image_source', 'image_url'],
        num_rows: 5157
    })
})


In [22]:
ds = ds['test']

## 1.APC Pipeline

In [5]:
# import os
# os.environ["PYOPENGL_PLATFORM"] = "osmesa"  # or "egl" if CUDA drivers support EGL
# os.environ["DISPLAY"] = ":0"

import os
os.environ["PYOPENGL_PLATFORM"] = "egl"   # use EGL instead of OSMesa for NVIDIA drivers
os.environ.pop("DISPLAY", None)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [6]:
!nvidia-smi

Fri Oct 24 10:52:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|


|   0  Tesla V100-SXM2-32GB           On  |   00000000:15:00.0 Off |                    0 |
| N/A   29C    P0             54W /  300W |   32412MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+------------------------+----------------------+
                                                                                         
+-----------------------------------------------------------------------------------------+
| Processes:                                                                              |
|  GPU   GI   CI        PID   Type   Process name                              GPU Memory |
|        ID   ID                                                               Usage      |
|=========================================================================================|
|    0   N/A  N/A     27064      C   ...inga/envs/apc_vlm_env/bin/python3.9      2

In [8]:
!kill -9 50536

In [9]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [10]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ["DISPLAY"] =':1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # set GPU device
sys.path.append("apc/vision_modules")
import yaml
import re
import requests
from box import Box
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from io import BytesIO
from PIL import Image
# import APC pipeline
from apc.apc_pipeline import APC
from apc.utils import visualize_conversation, create_image_with_text

# set device
device_vlm = "cuda:0"
device_vision = "cuda:0"

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
# load config
config_path = "apc/configs/qwenvl2_5_7b_instruct.yaml"
with open(config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config = Box(config)

# load APC pipeline
apc = APC(config, device_vlm=device_vlm, device_vision=device_vision)

[INFO] Loaded config Qwen/Qwen2.5-VL-7B-Instruct
[INFO] Loaded config hidden_size=3584, model_type=qwen2_5_vl


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
2025-10-24 10:59:47 INFO [apc.apc_pipeline] [APC] Initialized VLM model: qwenvl2_5


[INFO] Loaded model type: qwen2_5_vl
[INFO] Hidden size: 3584


final text_encoder_type: bert-base-uncased


2025-10-24 11:00:16 INFO [depth_pro.network.vit_factory] Module(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (norm): Identity()
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(

In [12]:
def download_image(image_url):
    response = requests.get(image_url, timeout=20)
    image = Image.open(BytesIO(response.content)).convert("RGB").resize((512, 512))
    return image

In [13]:
import pyrender
import trimesh
import numpy as np
from PIL import Image
import io

def path3d_to_trimesh(path, radius=0.002, color=[255, 255, 255, 255]):
    """
    Convert a trimesh.path.Path3D into a small-radius tube mesh for rendering.
    Handles both old/new Trimesh APIs (vertices vs points).
    """
    segments = []

    for entity in path.entities:
        # Find vertex indices robustly
        if hasattr(entity, "vertices"):
            idx = entity.vertices
        elif hasattr(entity, "points"):
            idx = entity.points
        else:
            continue

        # Ensure valid indices
        if len(idx) < 2:
            continue

        try:
            start = np.array(path.vertices[idx[0]])
            end = np.array(path.vertices[idx[1]])
        except Exception:
            continue

        # Skip degenerate lines
        if np.allclose(start, end):
            continue

        try:
            # NOTE: cylinder() expects two 3D points (shape (2,3))
            cyl = trimesh.creation.cylinder(radius=radius, segment=np.vstack([start, end]))
            cyl.visual.vertex_colors = np.tile(color, (len(cyl.vertices), 1))
            segments.append(cyl)
        except Exception as e:
            print(f"⚠️ Cylinder creation failed for Path3D segment: {e}")

    if not segments:
        return None

    return trimesh.util.concatenate(segments)
    
def save_image_with_pyrender(scene_or_mesh, resolution=(512, 512)):
    """
    Robust EGL-based renderer for Trimesh scenes using Pyrender.
    Works safely even if the scene includes Path3D, PointCloud, or face-colored meshes.
    Returns PNG bytes.
    """
    # Normalize input
    if isinstance(scene_or_mesh, trimesh.Trimesh):
        scene = trimesh.Scene(scene_or_mesh)
    else:
        scene = scene_or_mesh

    render_scene = pyrender.Scene()

    # Iterate through all geometries in the scene
    for name, geom in scene.geometry.items():
        if isinstance(geom, trimesh.Trimesh):
            try:
                # Determine if the mesh has per-face colors
                has_face_colors = (
                    hasattr(geom, "visual")
                    and hasattr(geom.visual, "face_colors")
                    and geom.visual.face_colors is not None
                    and len(geom.visual.face_colors) == len(geom.faces)
                )

                # Convert face colors → vertex colors for smooth shading
                if has_face_colors:
                    geom = geom.copy()
                    try:
                        geom.visual = geom.visual.to_vertex_colors()
                    except Exception:
                        # Some visuals may not support conversion cleanly
                        has_face_colors = True  # fallback to flat shading

                mesh_render = pyrender.Mesh.from_trimesh(
                    geom,
                    smooth=not has_face_colors
                )
                render_scene.add(mesh_render)

            except Exception as e:
                print(f"⚠️ Skipping mesh {name}: {e}")
        elif isinstance(geom, trimesh.path.Path3D):
            try:
                path_mesh = path3d_to_trimesh(geom, radius=0.002)
                if path_mesh is not None:
                    mesh_render = pyrender.Mesh.from_trimesh(path_mesh, smooth=False)
                    render_scene.add(mesh_render)
            except Exception as e:
                print(f"⚠️ Failed to render Path3D '{name}': {e}")
        else:
            print(f"⚠️ Skipping non-mesh geometry: {name} ({type(geom).__name__})")

    # Add a basic light
    light = pyrender.DirectionalLight(color=np.ones(3), intensity=3.0)
    render_scene.add(light, pose=np.eye(4))

    # Simple camera
    camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0)
    render_scene.add(camera, pose=[[1,0,0,0],
                                   [0,1,0,0],
                                   [0,0,1,2.5],
                                   [0,0,0,1]])

    # Offscreen render via EGL
    r = pyrender.OffscreenRenderer(*resolution)
    color, _ = r.render(render_scene)
    r.delete()

    # Return PNG bytes
    buf = io.BytesIO()
    Image.fromarray(color).save(buf, format="PNG")
    return buf.getvalue()

trimesh.Scene.save_image = save_image_with_pyrender

2025-10-24 11:03:29 INFO [OpenGL.acceleratesupport] No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


In [ ]:
# import warnings
# import torch

# # Suppress all general warnings
# warnings.filterwarnings("ignore")

# # Optionally, suppress only specific warnings
# warnings.filterwarnings("ignore", category=UserWarning)
# warnings.filterwarnings("ignore", category=FutureWarning)

# # For torch checkpoint warning specifically
# import torch.utils.checkpoint
# torch.utils.checkpoint.use_reentrant = False  # explicitly set new default (if needed)


In [ ]:
# import random

# random.seed(42)
# indices = random.sample(range(len(ds)), 100)

In [ ]:
# from datasets import concatenate_datasets

# # how many total samples to select
# total_samples = 180

# # unique categories
# categories = list(set(ds["category"]))
# samples_per_cat = total_samples // len(categories)

# balanced_subsets = []

# for cat in categories:
#     subset = ds.filter(lambda x: x["category"] == cat)
#     n = min(samples_per_cat, len(subset))  # handle small categories
#     subset = subset.shuffle(seed=42).select(range(n))
#     balanced_subsets.append(subset)

# # combine them and shuffle the final set
# balanced_ds = concatenate_datasets(balanced_subsets).shuffle(seed=42)

# print(f"✅ Balanced dataset size: {len(balanced_ds)}")
# print(balanced_ds['category'][:20])  # preview the categories


✅ Balanced dataset size: 180
['multi_object_parallel', 'orientation_viewpoint', 'multi_object_viewpoint_towards_object', 'orientation_viewpoint', 'multi_object_same_direction', 'orientation_in_front_of', 'orientation_on_the_left', 'multi_object_closer_to', 'multi_object_parallel', 'height_higher', 'location_above', 'orientation_viewpoint', 'orientation_on_the_left', 'multi_object_closer_to', 'orientation_in_front_of', 'multi_object_same_direction', 'orientation_viewpoint', 'location_closer_to_camera', 'height_higher', 'orientation_viewpoint']


In [ ]:
# balanced_ds.save_to_disk("test_dataset_150")

Saving the dataset (0/1 shards):   0%|          | 0/180 [00:00<?, ? examples/s]

In [14]:
from datasets import load_from_disk
ds = load_from_disk("test_dataset_150")

In [ ]:
# with open("test_dataset_150.txt", "w") as f:
#     for idx in balanced_ds["index"]:
#         f.write(str(idx) + "\n")

In [ ]:
# ds = balanced_ds

In [15]:
def run_single(i, example):
    image_url = example["image_url"]
    question = example["question"]
    options = [example["A"], example["B"], example["C"], example["D"]]
    correct = example["answer"]
    category = example["category"]

    # Download image
    image = download_image(image_url)

    # plt.imshow(image)
    # plt.axis("off")
    # plt.title(f"Index {i} — Category: {example['category']}")
    # plt.show()
    
    # Build prompt
    valid_options = [(chr(65 + j), opt) for j, opt in enumerate(options) if opt and str(opt).strip()]
    options_text = " or ".join([f"{label}. {opt}" for label, opt in valid_options])
    prompt = f"From the camera's point of view, {question.strip()} {options_text}, give the letter of the correct answer."

    # print("Prompt:")
    # print(prompt)

    # Directory for saving intermediate results
    save_dir = f"outputs/benchmark/{category}_{i}"
    os.makedirs(save_dir, exist_ok=True)

    image_with_text = create_image_with_text(image, "[Q] " + prompt, fontsize=20)
    # image_with_text.save(f"outputs/benchmark/3DSRBench_{i}_prompt.png")

    # Run APC pipeline
    response_text, conv_history = apc.run_apc(
        image,
        prompt,
        trace_save_dir=save_dir,
        perspective_prompt_type="visual",
        visualize_trace=False,
        visualize_scene_abstraction=False,
        return_conv_history=False,
        logging=False,
    )

    # print("Response", response_text)
    # Extract predicted answer (search for 'A', 'B', 'C', 'D')
    match = re.search(r"\b([ABCD])\b", response_text.upper())
    pred_letter = match.group(1) if match else None

    # print(len(conv_history))
    # for i in range(len(conv_history)):
        # print(conv_history[i])

    # conv_viz = visualize_conversation(
    #     conv_history,
    #     width=900,
    #     row_gap=0,
    #     font_size=13,
    #     image_max_width=180,
    #     output_path=os.path.join(save_dir, "conversation_viz.png")
    # )

    return {
        "index": example["index"],
        "category": category,
        "question": question,
        "prediction": pred_letter,
        "answer": correct,
        "is_correct": pred_letter == correct,
        "response_text": response_text,
    }

In [16]:
def run():
    results = []

    for i, example in enumerate(tqdm(ds, desc="Evaluating 3DSRBench")):
        results.append(run_single(i, ds[i]))
    
    return results

In [17]:
print(ds)

Dataset({
    features: ['index', 'question', 'A', 'B', 'C', 'D', 'answer', 'category', 'image_source', 'image_url'],
    num_rows: 180
})


In [ ]:
# results = run_single(0, ds[0])
# results = run()
# df = pd.DataFrame(results)

In [18]:
for i in range(5):
    results = run()
    df = pd.DataFrame(results)
    df.to_csv(f"3DSRBench_raw_predictions_{i}.csv", index=False)

# !scancel -u vwei

Evaluating 3DSRBench:   1%|                    | 1/180 [00:42<2:06:51, 42.52s/it]


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results)
df.to_csv("3DSRBench_raw_predictions.csv", index=False)

## Compute metrics

In [39]:
df = pd.read_csv("3DSRBench_raw_predictions_5.csv")

In [40]:
metrics = {}

# ---- Overall accuracy ----
metrics["overall_accuracy"] = df["is_correct"].mean()

# # ---- Standard deviation of correctness (robustness) ----
# metrics["std_dev_accuracy"] = df["is_correct"].std()

# ---- Category-wise accuracy ----
cat_acc = df.groupby("category")["is_correct"].mean().to_dict()
for cat, acc in cat_acc.items():
    metrics[f"acc_{cat}"] = acc

In [41]:

# =======================
# Save results
# =======================
metrics_df = pd.DataFrame([metrics])
metrics_df.to_csv("3DSRBench_metrics_summary_4.csv", index=False)

print("\n✅ Evaluation Complete!")
print(metrics_df.T)



✅ Evaluation Complete!
                                                  0
overall_accuracy                           0.455556
acc_height_higher                          0.400000
acc_location_above                         0.733333
acc_location_closer_to_camera              0.200000
acc_location_next_to                       0.666667
acc_multi_object_closer_to                 0.333333
acc_multi_object_facing                    0.133333
acc_multi_object_parallel                  0.600000
acc_multi_object_same_direction            0.666667
acc_multi_object_viewpoint_towards_object  0.266667
acc_orientation_in_front_of                0.666667
acc_orientation_on_the_left                0.533333
acc_orientation_viewpoint                  0.266667
